<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/cnn_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [85]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 48.3 MB/s 
     |████████████████████████████████| 144 kB 47.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


In [86]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [87]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import pandas as pd
import numpy as np

In [88]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [89]:
class ImageDataLoader(Dataset):
    def __init__(self, dir_=None):
        self.data_df = pd.read_csv('gdrive/MyDrive/data.csv')
        self.dataset_len = len(self.data_df) # read the number of len of your csv files
    def __getitem__(self, idx):
        # load the next image
        f_name_t = self.data_df['Filename'][idx]
        f_name_tp1 = self.data_df['Filename'][idx+1]
        label = self.data_df['Label'][idx]
        label = label.astype(np.float32) 
        label = np.true_divide(label, 10)
        img_t = torchvision.io.read_image('gdrive/MyDrive/trainingData/{}'.format(f_name_t))
        img_tp1 = torchvision.io.read_image('gdrive/MyDrive/trainingData/{}'.format(f_name_tp1))
        img_t = img_t.float().div_(255.0)
        img_tp1 = img_tp1.float().div_(255.0)
        return img_t, img_tp1, label
    def __len__(self):
        return self.dataset_len - 1

In [90]:
dataset = ImageDataLoader()
dataloader = DataLoader(dataset, shuffle=False, batch_size=8)

In [91]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 8, 1, 1),
            nn.ReLU(),
            nn.Conv2d(8, 16, 1, 1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc1 = nn.Linear(65536, 128)
    def forward(self, x):
        state = self.cnn(x)
        state = self.fc1(state)
        # print('size of the state after CNN ',state.size())
        return state

In [92]:
encoder = Encoder().to(device)

In [93]:
batch_size = 8
num_epochs = 1000
learning_rate = 0.0009
input_size = 128
output_size = 1
sequence_length = 2
hidden_size = 128
num_layers = 1

In [94]:
class RNN(nn.Module):
    def __init__(self, batch_size, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.rnn = nn.RNN(self.input_size, self.hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device))
    def forward(self, x):
        self.batch_size = x.size(0)
        self.hidden = self.init_hidden()
        out, self.hidden = self.rnn(x, self.hidden)
        out = self.fc(out)
        return out

In [95]:
model = RNN(batch_size=batch_size, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size).to(device)

In [96]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [97]:
wandb.init(project="test-project", entity="individual_project")

wandb: Currently logged in as: individual_project (use `wandb login --relogin` to force relogin)


In [98]:
wandb.config = {
  "learning_rate": learning_rate,
  "epochs": num_epochs,
  "batch_size": batch_size
}

In [100]:
n_total_steps = len(dataloader)
for epoch in range(num_epochs):
    runningLoss = 0
    for i, (image1, image2, label) in enumerate(dataloader):
        output1 = encoder(image1.to(device))
        output2 = encoder(image2.to(device))
        # print('o1 shape:', output1.size())
        batch_size1 = len(output1)
        batch_size2 = len(output2)
        output1 = output1.reshape(batch_size1,1,-1)
        output2 = output2.reshape(batch_size2,1,-1)
        # print('o1 after shape:', output1.size())
        seq = torch.cat((output1, output2.detach()), dim=1)
        # print('seq size: ', seq.size())
        label = (label.float())
        # Forward pass
        outputs = model(seq.to(device))
        loss = criterion(outputs[:,-1].squeeze(), label.to(device))
        wandb.log({"loss": loss})
        # print(outputs[:,-1].squeeze())
        # print(label)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningLoss += loss
    if epoch%10 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Running Loss: {runningLoss:.4f}, Average Loss: {runningLoss/len(dataloader.dataset):.4f}')
    runningLoss = 0
wandb.finish()

Epoch [1/1000], Running Loss: 127.7710, Average Loss: 0.5090
Epoch [11/1000], Running Loss: 362.7186, Average Loss: 1.4451
Epoch [21/1000], Running Loss: 362.0891, Average Loss: 1.4426
Epoch [31/1000], Running Loss: 361.0649, Average Loss: 1.4385
Epoch [41/1000], Running Loss: 360.7513, Average Loss: 1.4373
Epoch [51/1000], Running Loss: 360.6877, Average Loss: 1.4370
Epoch [61/1000], Running Loss: 358.7665, Average Loss: 1.4293
Epoch [71/1000], Running Loss: 360.0424, Average Loss: 1.4344
Epoch [81/1000], Running Loss: 359.1077, Average Loss: 1.4307
Epoch [91/1000], Running Loss: 358.9489, Average Loss: 1.4301
Epoch [101/1000], Running Loss: 365.8826, Average Loss: 1.4577
Epoch [111/1000], Running Loss: 357.1895, Average Loss: 1.4231
Epoch [121/1000], Running Loss: 357.8010, Average Loss: 1.4255
Epoch [131/1000], Running Loss: 358.6025, Average Loss: 1.4287
Epoch [141/1000], Running Loss: 359.5802, Average Loss: 1.4326
Epoch [151/1000], Running Loss: 349.5533, Average Loss: 1.3926
Epo

loss,██████▇▇▆▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.03458
